In [1]:
'''
## Reddit API:
1. Go to https://www.reddit.com/prefs/apps
2. Click "create application"
3. Name: My Sentiment Example
4. Choose script
5. Leave *description* and *about url* blank
6. Add http://www.example.com/unused/redirect/uri to *redirect uri*
'''

import openai
import os
import praw

In [2]:
os.environ['reddit_client_id']='67zYzfBPiq5IBSG3s2svnA'
os.environ['reddit_client_secret']='tmv6DbqdvWI234D6n64zRAS_8Olj8Q'

In [3]:
reddit=praw.Reddit(client_id=os.getenv("reddit_client_id"), 
                   client_secret=os.getenv("reddit_client_secret"), 
                   user_agent='Sentiment Analysis')

In [4]:
'''
We can now already use our reddit instance to obtain public information from reddit!
'''
for submission in reddit.subreddit('finance').hot(limit=5):
    print(submission.title)

Moronic Monday - August 08, 2023 - Your Weekly Questions Thread
We are Wall Street Journal reporters covering the collapse of SVB. Ask us anything.
China’s property crisis deepens as another huge developer risks default
China tips into deflation as efforts to stoke recovery falter
Moody's downgrades US banks, warns of possible cuts to others


In [5]:
subreddit_stocks = reddit.subreddit("stocks")

In [6]:
print(subreddit_stocks.display_name)
print(subreddit_stocks.title)
print(subreddit_stocks.accounts_active)

stocks
Stocks - Investing and trading for all
4647


In [7]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)

Rate My Portfolio - r/Stocks Quarterly Thread June 2023
r/Stocks Daily Discussion & Options Trading Thursday - Aug 10, 2023
Penn says that it expects to record a loss of between $800 million-$850 million on Barstool Sports. As Dave Portnoy buys it back for $1.
Insiders keep selling Apple stocks and no one buying this year. Thoughts?
Disney posts mixed results for quarter plagued by streaming woes, restructuring costs


In [8]:
for post in subreddit_stocks.hot(limit=5):
    print(post.id)

13xchbr
15n7irb
15mvtnn
15mwwsd
15mqzak


In [9]:
for post in subreddit_stocks.hot(limit=5):
    # Get title for each post
    print(post.title)
    # Get comments for each post
    submission = reddit.submission(post.id)
    c = 0
    #  Get comments for each post and print them out for top 2 comments
    for comment in submission.comments:
        comment = comment.body
        if not comment == "[deleted]":
            print(comment)
            c+=1
        if c == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread June 2023
ABBV- AbbVie Inc

APD- Air Products & Chemicals, Inc.

GOOGL- Alphabet Inc Class A

AMZN - Amazon.com, Inc.

AWK - American Water Works Company Inc

ADSK - Autodesk Inc

BRK.B - Berkshire Hathaway Inc Class B

CHTR - Charter Communications Inc

COST - Costco Wholesale Corporation

CCI -Crown Castle Inc

DG - Dollar General Corp

HSY - Hershey Co

INTC - Intel Corporation

IP - International Paper Co

LOW - Lowe's Companies Inc

MA - Mastercard Inc

MSFT - Microsoft Corp

PGR - Progressive Corp

PLD - Prologis Inc

PSA - Public Storage

VALE - Vale SA

WBA - Walgreens Boots Alliance Inc

WM - Waste Management, Inc.
Let me know what you think about my portfolio. Trying to stay away from the sectors I don't know and increase diversity by investing VOO and QQQ ETFs  


17.39% AAPL

13.86% GOOGL

26.31% VOO

12.63% BAC

04.99% TQQQ

11.47% AMZN

13.37% QQQM
r/Stocks Daily Discussion & Options Trading Thursday - Aug 10, 2023
AIT earning

In [11]:
#
getattr(subreddit_stocks, "hot")

<bound method BaseListingMixin.hot of Subreddit(display_name='stocks')>

In [10]:
# get titles and comments from reddit
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=5, num_comments=2, skip_first=2):
    
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):
        # skip first 2 posts
        if c < skip_first:
            continue
        # adjust counter to account for skipped posts
        c+=(1-skip_first)
        
        titles_and_comments[c] = ""
        # get submission
        submission = reddit.submission(post.id)
        # get title
        title = post.title
    
        titles_and_comments[c] += "Title: " + title  + "\n\n"
        titles_and_comments[c] += "Comments: \n\n"
        
        
        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[c] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments

In [12]:
titles_and_comments = get_titles_and_comments(subreddit="stocks", limit=5)

In [13]:
print(titles_and_comments[2])

Title: Insiders keep selling Apple stocks and no one buying this year. Thoughts?

Comments: 

How many shares does SHE still have? I don’t think it means anything. She’s been with apple since 1988. You gotta cash out out some shares every now and then to pay the bills or buy a boat
Companies that grant stock to employees tend not to have insider buys.



In [14]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments

In [15]:
prompt = create_prompt(titles_and_comments[1])

In [16]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: Penn says that it expects to record a loss of between $800 million-$850 million on Barstool Sports. As Dave Portnoy buys it back for $1.

Comments: 

Having ESPN involved with gambling seems really odd.
They structured it to get a tax loss against the asset.

But if portnoy sells it again; they get half of gross.



In [17]:
openai.api_key=os.getenv("OPENAI_API_KEY")

In [19]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment: " + response["choices"][0]["text"])
    print("-"*100)

Title: Penn says that it expects to record a loss of between $800 million-$850 million on Barstool Sports. As Dave Portnoy buys it back for $1.

Comments: 

Having ESPN involved with gambling seems really odd.
They structured it to get a tax loss against the asset.

But if portnoy sells it again; they get half of gross.

Sentiment: 
Ticker/Company: Penn 
Sentiment: Neutral
----------------------------------------------------------------------------------------------------
Title: Insiders keep selling Apple stocks and no one buying this year. Thoughts?

Comments: 

How many shares does SHE still have? I don’t think it means anything. She’s been with apple since 1988. You gotta cash out out some shares every now and then to pay the bills or buy a boat
Companies that grant stock to employees tend not to have insider buys.

Sentiment: 
Company: Apple
Sentiment: Neutral
----------------------------------------------------------------------------------------------------
Title: Disney posts m